# Fine-Tune Whisper For Multilingual ASR with 🤗 Transformers

We can verify that we've been assigned a GPU and view its specifications:
> This code runs nvidia-smi to check GPU status. If the command fails, it prints "Not connected to a GPU." If successful, it prints the GPU information.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

This block of code is used to upgrade and install Python packages using `pip`. Here's what each line does:

1. **`!pip install --upgrade pip`**: Upgrades the `pip` package manager to the latest version.

2. **`!pip install --upgrade torch`**: Upgrades the `torch` package (PyTorch) to the latest version.

3. **`!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio`**: Upgrades or installs several packages:
   - `datasets[audio]`: The `datasets` library with audio support.
   - `transformers`: The Hugging Face Transformers library for natural language processing.
   - `accelerate`: A library for efficient training of machine learning models.
   - `evaluate`: A library for evaluating machine learning models.
   - `jiwer`: A library for evaluating speech-to-text transcription accuracy.
   - `tensorboard`: A tool for visualizing machine learning experiments.
   - `gradio`: A library for creating user interfaces for machine learning models.

The `!` at the beginning indicates these commands are intended to be run in a Jupyter notebook or similar environment.

In [ ]:
!pip install --upgrade pip
!pip install --upgrade torch
!pip install --upgrade datasets[audio] transformers accelerate evaluate jiwer tensorboard gradio

This code snippet imports several modules and functions necessary for working with datasets and the Whisper model from Hugging Face:

1. **`from datasets import load_dataset, DatasetDict`**: 
   - `load_dataset`: Function to load datasets from the Hugging Face Hub or local files.
   - `DatasetDict`: A class representing a dictionary of datasets.

2. **`from huggingface_hub import notebook_login`**: Imports `notebook_login`, a function for authenticating with the Hugging Face Hub in a Jupyter notebook.

3. **`from transformers import WhisperProcessor`**: Imports `WhisperProcessor`, which is used to process audio inputs for the Whisper model.

4. **`from transformers import WhisperTokenizer`**: Imports `WhisperTokenizer`, which is used to tokenize text for the Whisper model.

5. **`from transformers import WhisperFeatureExtractor`**: Imports `WhisperFeatureExtractor`, which is used to extract features from audio data for the Whisper model.

6. **`from datasets import Audio`**: Imports the `Audio` class from the `datasets` library, which is used for handling audio data.


In [ ]:
from datasets import load_dataset, DatasetDict
from huggingface_hub import notebook_login
from transformers import WhisperProcessor
from transformers import WhisperTokenizer
from transformers import WhisperFeatureExtractor
from datasets import Audio

`notebook_login()` authenticates your Hugging Face account in a Jupyter notebook, allowing access to private models and datasets.

In [ ]:
notebook_login() 

# *Load Dataset:*

This code snippet does the following:

1. **Creates an empty `DatasetDict` object**: `common_voice = DatasetDict()`.

2. **Loads datasets into the `DatasetDict`**:
   - **`common_voice["train"]`**: Loads the training and validation data from the Common Voice dataset (version 11.0, Urdu language) and assigns it to the "train" key.
   - **`common_voice["test"]`**: Loads the test data from the same dataset and assigns it to the "test" key.

3. **Prints the `common_voice` object**: Displays the contents of the `DatasetDict`, showing the structure and data of the loaded datasets.

In [ ]:
common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "ur", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "ur", split="test", use_auth_token=True)

print(common_voice)

Removes specified columns from the `common_voice` `DatasetDict` and then prints the updated object. Here's what each line does:

1. **`common_voice = common_voice.remove_columns([...])`**: Removes the columns listed from the `common_voice` `DatasetDict`. The columns being removed are `"accent"`, `"age"`, `"client_id"`, `"down_votes"`, `"gender"`, `"locale"`, `"path"`, `"segment"`, and `"up_votes"`.

Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:


In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:

1. A feature extractor which pre-processes the raw audio-inputs
2. The model which performs the sequence-to-sequence mapping
3. A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Hindi, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [ ]:
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="ur", task="transcribe")

### Combine To Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="ur", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
print(common_voice["train"][0])

Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many CPU cores to use. Setting `num_proc` > 1 will enable multiprocessing. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

## Training and Evaluation

Now that we've prepared our data, we're ready to dive into the training pipeline.
The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)
will do much of the heavy lifting for us. All we have to do is:

- Load a pre-trained checkpoint: we need to load a pre-trained checkpoint and configure it correctly for training.

- Define a data collator: the data collator takes our pre-processed data and prepares PyTorch tensors ready for the model.

- Evaluation metrics: during evaluation, we want to evaluate the model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We need to define a `compute_metrics` function that handles this computation.

- Define the training configuration: this will be used by the 🤗 Trainer to define the training schedule.

Once we've fine-tuned the model, we will evaluate it on the test data to verify that we have correctly trained it
to transcribe speech in Hindi.

### Load a Pre-Trained Checkpoint

We'll start our fine-tuning run from the pre-trained Whisper `small` checkpoint,
the weights for which we need to load from the Hugging Face Hub. Again, this
is trivial through use of 🤗 Transformers!

In [ ]:
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

We can disable the automatic language detection task performed during inference, and force the model to generate in Hindi. To do so, we set the [langauge](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.language)
and [task](https://huggingface.co/docs/transformers/en/model_doc/whisper#transformers.WhisperForConditionalGeneration.generate.task)
arguments to the generation config. We'll also set any [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)
to None, since this was the legacy way of setting the language and
task arguments:

In [ ]:
model.generation_config.language = "ur"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

### Define a Data Collator

The data collator for a sequence-to-sequence speech model is unique in the sense that it
treats the `input_features` and `labels` independently: the  `input_features` must be
handled by the feature extractor and the `labels` by the tokenizer.

The `input_features` are already padded to 30s and converted to a log-Mel spectrogram
of fixed dimension by action of the feature extractor, so all we have to do is convert the `input_features`
to batched PyTorch tensors. We do this using the feature extractor's `.pad` method with `return_tensors=pt`.

The `labels` on the other hand are un-padded. We first pad the sequences
to the maximum length in the batch using the tokenizer's `.pad` method. The padding tokens
are then replaced by `-100` so that these tokens are **not** taken into account when
computing the loss. We then cut the BOS token from the start of the label sequence as we
append it later during training.

We can leverage the `WhisperProcessor` we defined earlier to perform both the
feature extractor and the tokenizer operations:

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id, 
   
)

### Evaluation Metrics

We'll use the word error rate (WER) metric, the 'de-facto' metric for assessing
ASR systems. For more information, refer to the WER [docs](https://huggingface.co/metrics/wer). We'll load the WER metric from 🤗 Evaluate:

In [ ]:
metric = evaluate.load("wer")

We then simply have to define a function that takes our model
predictions and returns the WER metric. This function, called
`compute_metrics`, first replaces `-100` with the `pad_token_id`
in the `label_ids` (undoing the step we applied in the
data collator to ignore padded tokens correctly in the loss).
It then decodes the predicted and label ids to strings. Finally,
it computes the WER between the predictions and reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Define the Training Configuration

Initializes the `Accelerator` for distributed and mixed-precision training, then sets up training arguments for a sequence-to-sequence model using the `Seq2SeqTrainingArguments` class. Here’s a brief overview:

1. **Initialize Accelerator**:
   - **`accelerator = Accelerator()`**: Creates an `Accelerator` instance, which helps manage distributed training, mixed-precision, and other training aspects.

2. **Training Arguments**:
   - **`output_dir="./WhisperLiveSubs"`**: Directory where the model and checkpoints will be saved.
   - **`per_device_train_batch_size=8`**: Batch size per device for training.
   - **`gradient_accumulation_steps=2`**: Accumulates gradients over 2 steps before updating the model weights.
   - **`learning_rate=1e-5`**: Learning rate for the optimizer.
   - **`warmup_steps=500`**: Number of warmup steps for learning rate scheduling.
   - **`max_steps=4000`**: Maximum number of training steps.
   - **`gradient_checkpointing=True`**: Enables gradient checkpointing to save memory.
   - **`fp16=True`**: Enables mixed-precision training (16-bit floating-point).
   - **`evaluation_strategy="steps"`**: Evaluates the model at regular steps.
   - **`per_device_eval_batch_size=4`**: Batch size per device for evaluation.
   - **`predict_with_generate=True`**: Enables generation during prediction.
   - **`generation_max_length=225`**: Maximum length of generated sequences.
   - **`save_steps=1000`**: Saves model checkpoints every 1000 steps.
   - **`eval_steps=1000`**: Evaluates the model every 1000 steps.
   - **`logging_steps=25`**: Logs training metrics every 25 steps.
   - **`report_to=["tensorboard"]`**: Reports metrics to TensorBoard.
   - **`load_best_model_at_end=True`**: Loads the best model based on evaluation metrics at the end of training.
   - **`metric_for_best_model="wer"`**: Uses Word Error Rate (WER) to determine the best model.
   - **`greater_is_better=False`**: Indicates that lower WER is better.
   - **`push_to_hub=True`**: Pushes the trained model to the Hugging Face Hub.

In the final step, we define all the parameters related to training. For more detail on the training arguments, refer to the Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).


In [ ]:
#Initialize Accelerator
accelerator = Accelerator()

# Use it in the training script
training_args = Seq2SeqTrainingArguments(
    output_dir="./WhisperLiveSubs", 
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=4,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=True,
)


**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can forward the training arguments to the 🤗 Trainer along with our model,
dataset, data collator and `compute_metrics` function:


Initializes a `Seq2SeqTrainer` for training a sequence-to-sequence model. Here's a breakdown:

1. **`Seq2SeqTrainer` Initialization**:
   - **`args=training_args`**: Passes the training arguments defined earlier.
   - **`model=model`**: Specifies the model to be trained.
   - **`train_dataset=common_voice["train"]`**: Uses the training dataset from the `common_voice` `DatasetDict`.
   - **`eval_dataset=common_voice["test"]`**: Uses the test dataset from the `common_voice` `DatasetDict`.
   - **`data_collator=data_collator`**: Specifies the data collator function or object that handles batching and padding.
   - **`compute_metrics=compute_metrics`**: Provides a function to compute evaluation metrics.
   - **`tokenizer=processor.tokenizer`**: Uses the tokenizer from the `processor` object to tokenize inputs and outputs.

This sets up the `Seq2SeqTrainer` with all the necessary components for training and evaluating the sequence-to-sequence model.

In [ ]:
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.tokenizer,
)

We'll save the processor object once before starting training. Since the processor is not trainable, it won't change over the course of training:

In [ ]:
processor.save_pretrained(training_args.output_dir)

# Training

## For Colab


Training will take approximately 5-10 hours depending on your GPU or the one
allocated to this Google Colab. If using this Google Colab directly to
fine-tune a Whisper model, you should make sure that training isn't
interrupted due to inactivity. A simple workaround to prevent this is
to paste the following code into the console of this tab (_right mouse click_
-> _inspect_ -> _Console tab_ -> _insert code_).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The peak GPU memory for the given training configuration is approximately 15.8GB.
Depending on the GPU allocated to the Google Colab, it is possible that you will encounter a CUDA `"out-of-memory"` error when you launch training.
In this case, you can reduce the `per_device_train_batch_size` incrementally by factors of 2
and employ [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps)
to compensate.


## Kaggle

I recommend using Kaggle.

The `Seq2SeqTrainer` for training using the `Accelerator` and then starts the training process:

1. **`trainer = accelerator.prepare(trainer)`**: Prepares the `trainer` for distributed and mixed-precision training using the `Accelerator`. This step ensures that the trainer and its components (model, optimizer, etc.) are set up correctly for the training environment.

2. **`trainer.train()`**: Starts the training process with the prepared trainer. This will execute the training loop based on the provided training arguments and datasets.

In [ ]:
# Use accelerator with trainer
trainer = accelerator.prepare(trainer)
trainer.train()

Our best WER is 32.0% - not bad for 8h of training data! We can make our model more accessible on the Hub with appropriate tags and README information.
You can change these values to match your dataset, language and model
name accordingly:

In [ ]:
dict_ = {
    "dataset_tags": "mozilla-foundation/common_voice_11_0",
    "dataset": "Common Voice 11.0",  # a 'pretty' name for the training dataset
    "dataset_args": "config: hi, split: test",
    "language": "ur",
    "model_name": "WhisperLiveSubs",  # a 'pretty' name for our model
    "finetuned_from": "openai/whisper-small",
    "tasks": "automatic-speech-recognition",
}

The training results can now be uploaded to the Hub. To do so, execute the `push_to_hub` command and save the preprocessor object we created:

In [ ]:
trainer.push_to_hub(**dict_)

## Building a Demo

Now that we've fine-tuned our model we can build a demo to show
off its ASR capabilities! We'll make use of 🤗 Transformers
`pipeline`, which will take care of the entire ASR pipeline,
right from pre-processing the audio inputs to decoding the
model predictions.

Running the example below will generate a Gradio demo where we
can record speech through the microphone of our computer and input it to
our fine-tuned Whisper model to transcribe the corresponding text:

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="codewithdark/WhisperLiveSubs")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    inputs=gr.Audio("microphone", type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch(share=True)

## Closing Remarks

In this blog, we covered a step-by-step guide on fine-tuning Whisper for multilingual ASR
using 🤗 Datasets, Transformers and the Hugging Face Hub. For more details on the Whisper model, the Common Voice dataset and the theory behind fine-tuning, refere to the accompanying [blog post](https://huggingface.co/blog/fine-tune-whisper). If you're interested in fine-tuning other
Transformers models, both for English and multilingual ASR, be sure to check out the
examples scripts at [examples/pytorch/speech-recognition](https://github.com/huggingface/transformers/tree/main/examples/pytorch/speech-recognition).